<img style="float: center;" src="images/CI_horizontal.png" width="600">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

<center> Julia Lane, Benjamin Feder, Angela Tombari, Ekaterina Levitskaya, Tian Lou, Lina Osorio-Copete. </center> 

# Dataset Exploration

## Table of Contents

JupyterLab contains a dynamic table of contents that can be accessed by clicking the sixth (second from bottom) icon on the left-hand toolbar.

## Introduction

In an ideal world, we have all of the data we want with all of the desirable properties (no missing values, no errors, standard formats, and so on). We'd also have perfect data documentation, with summary statistics and appropriate aggregate measures of everything we'd want to investigate. However, that is hardly ever true, and we have to use our datasets to answer questions of interest as intelligently as possible. 

In this notebook, we will discover the datasets we have on the ADRF and use them to answer some questions of interest.

### Learning Objectives

This notebook will give you the opportunity to spend some hands-on time with the data. 

Throughout the notebook, you will work through various techniques of how to use SQL and R to explore the various datasets in the ADRF and better understand what you are working with. This will form the basis of all the other types of analyses you will do in this class and is a crucial first step for any data analysis workflow. As you work through the notebook, we will have checkpoints for you try out your own code, but you can also think about how you might apply any of the techniques and code presented with other datasets as well. 

Throughout this series of Jupyter notebooks, we will work to answer one underlying question: **What are the earnings and employment outcomes for individuals who graduated from Kentucky's postsecondary education programs?**

We will analyze this question through a variety of different lenses and will start working toward a better understanding of the answer in this notebook. We will do so by defining a specific cohort of Kentucky graduates in the 2012-2013 academic year (2013 AY) and we will track their earnings outcomes over time. This exploration of the supply side of the labor market will later be supplemented by analysis of the demand side to enhance our understanding of the overall labor market.

We are going to show just a portion of what you might be interested in investigating to answer this overarching question, so don't feel restricted by the questions we've decided to try to answer.

>**When defining your research question(s), recall that one key benefit of working with Kentucky postsecondary graduates in the ADRF is the ability to leverage employment and wage information from border states.**

### Datasets We Will Explore In This Notebook
- **Kentucky Postsecondary Education Data System (KPEDS) data**: Comprehensive Kentucky data on postsecondary enrollments, coursework, grades, and degrees earned for all in-state postsecondary students who attend a 4-year Private (Independent), 2-year Public (KCTCS), and/or 4-year Public (both Research and Comprehensive) institution. In-state proprietary and out-of-state institutions are not covered in this system.
- **Kentucky Unemployment Insurance (UI) Wage data**: Kentucky workers' quarterly earnings and employment. 

**This notebook will provide an introduction and examples for:**

- How to create new tables from the larger tables in a database (sometimes called the "analytical frame")
- How to explore different variables of interest
- How to create aggregate metrics
- How to join tables
- How to generate descriptive statistics to describe a specific cohort

### Methods

With our SQL queries, we will:

- Learn about rows and columns in the data with queries using `SELECT`, `DISTINCT` and `ORDER BY` 
- Select subsets of tables from the database with `WHERE`
- Aggregate data over groups with `GROUP BY`

And we will use R to:

- Find group-based measures using `group_by` and `summarize`
- Sort values with `arrange` and `desc`
- Create new variables with `mutate`
- And so much more!

### R Setup

Before we can use R functions that are not available in `base` R, we need to load them using the built-in function `library()`. For example, running `library(tidyverse)` loads the `tidyverse` suite of packages.

> When you run the following code cell, don't worry about the message below. We will explain some of its output later.

In [ ]:
#database interaction imports
library(DBI)
library(RPostgreSQL)

# for data manipulation/visualization
library(tidyverse)

# scaling data, calculating percentages, overriding default graphing
library(scales)

__When in doubt, use shift + tab to read the documentation of a method. Full documentation can be printed with `?<package/function_name>`, e.g. `?tidyverse/ggplot` or `?sprintf`.__ Do not worry about memorizing the information in the help documentation - you can always run this command when you are unsure of how to use a function.

> Certain functions exist across multiple packages (e.g. the function `lag` exists in both the `dplyr` and `stats` package - also noted in the message yielded from `library(tidyverse)`. When calling a function, you can put the package name first to ensure that you are using the right one. For example, `dplyr::lag` or `stats::lag` calls the `lag` function from `dplyr` or `stats`, respectively. 

In [ ]:
# see help documentation for writeLines, a function we will use to check that our sql query is written as intended
?writeLines

## Load the Data

Since we are working with the PostgreSQL database `appliedda` in this course, we will demonstrate how to use R to read data from a relational database. First, we need to connect to `appliedda`.

### Establish a Connection to the Database

We will create the database connection using the `DBI`  and `RPostgreSQL` libraries. Each time you create a new notebook in this course, make sure you copy the following code chunk so you can connect to the database.

__Database Connection__

In [ ]:
# create an RPostgreSQL driver
drv <- dbDriver("PostgreSQL")

# connect to the database
con <- dbConnect(drv,dbname = "postgresql://stuffed.adrf.info/appliedda")

### Formulate Data Query

Next, we need to dictate what we want to pull in from `appliedda`. This part is similar to writing a SQL query in DBeaver. Depending on the data we are interested in, we can use different queries to pull different data. In this example, we will pull in 10 rows of Kentucky degree completions data, which is stored in the `kpeds_degree` table inside the `kystats_2020` schema.

__create a query as a `character` string object in R__

In [ ]:
# Create query character string
query <- "
SELECT *
FROM kystats_2020.kpeds_degree
LIMIT 10;
"

Here, we use `LIMIT` to read in only the first 10 rows because we're just looking to preview the data and we don't want to eat up memory by reading a huge data frame into R. 

> `LIMIT` provides one simple way to get a "sample" of data; however, using `LIMIT` does **not provide a _random_** sample. Data displayed using just the `LIMIT` clause will depend on what is fastest for the database to return.

You can see that we've just assigned our query to a character string in R.

In [ ]:
# print with newlines instead of /n
writeLines(query)

### Reading in the Data 

Now we can use `con` and `query` as inputs to `dbGetQuery()` to read the data into an R data frame. 

> Recall that `con` (our PostgreSQL connection) includes a reference that tells it what driver to use `drv`. Forgetting to set up `drv` would cause an error. If you take another look at the code establishing the connection to `appliedda`, you will see that the first line of code creates our driver.

In [ ]:
# read in data frame and assign to df
df <- dbGetQuery(con,query)

In [ ]:
# see first few rows of df
head(df)

### What is in the Database?

We will start our exploration by looking at the contents of our database. We will find the list of schemas in the database, the list of tables in these schemas, and the list of columns in these tables.


We will find this information from a built-in schema called `information_schema`, which provides metadata on the database. The `information_schema` schema has tables `tables`, `schemata`, and others, as you will see.

In general, as in DBeaver, database schemas and tables are denoted as `<schema_name>.<table_name>`. 

You will query the `information_schema` tables: 

- `information_schema.tables` for metadata on tables   
- `information_schema.schemata` for metadata on schemas   
- `information_schema.columns` for metadata on columns 

In [ ]:
# see 10 of the available schemas
query <- "
SELECT DISTINCT * 
FROM information_schema.schemata
LIMIT 10;
"
dbGetQuery(con,query)

As a reminder, in this class you have read access to the following schemas: `public`, `data_ohio_olda_2018`, `il_des_kcmo`, `kcmo_lehd`, `mo_dhe`,`in_dwd`, `in_data_2019`, `kystats_2020` and `ada_ky_20`. You only have write access to the `ada_ky_20` schema.

Let's see all of the tables that exist in the `kystats_2020` schema.

In [ ]:
# see all tables in the kystats_2020 schema
qry <- "
SELECT DISTINCT table_name 
FROM information_schema.tables 
WHERE table_schema IN ('kystats_2020')
"

dbGetQuery(con,qry)

<font color=red><h3> Checkpoint 1: Explore Columns </h3></font> 

Take a look at the columns in the `kpeds_degree` table in the `kystats_2020` schema. What are some of the variables that you might be able to use to define our cohort of 2013 academic year graduates?

> Refer to the data dictionary on the class website to understand what the different variables refer to.

In [ ]:
query <- "
SELECT * 
FROM information_schema.columns 
where table_schema = '__' and table_name = '__';
"

dbGetQuery(con,query)

### `sprintf`

We'll quickly cover the `sprintf` funtion, which we'll use to create our first flexible, or *parameterized*, query.

`sprintf` is a string manipulation function that enables us to use symbols as placeholders in R so we can interchange values in an expression. In this case, we would like to see the columns in the other schemas and tables provided to us. Rather than rewriting all the queries, we can use `sprintf` to parameterize the queries, making them much more flexible. 

`sprintf` takes the form:
`sprintf(base_string, input_parameter)`

The base string should have a placeholder to denote the part of the string that will be replaced by `input_parameter`.     
- `%s` placeholder where the replacement is a string   
- `%d`placeholder where the replacement is an integer

Let's take a simple example to see how `sprintf` works to see the variables in the `kpeds_degree` table in the `kystats_2020` schema using our query from Checkpoint 1. This time through, we'll limit the amount of columns displayed to 5.

In [ ]:
# Column names in kystats_2020.kpeds_degree

# assign schema and table here
schema <- "kystats_2020"
tbl <- "kpeds_degree"

# use base query and assign table_schema and table_name to '%s'
base_query <- "SELECT * 
FROM information_schema.columns 
WHERE table_schema = '%s' AND table_name = '%s'
limit 5"

# feed in the new schema and tbl to query
qry <- sprintf(base_query,schema, tbl)

# see query
writeLines(qry)

In [ ]:
# query from database
dbGetQuery(con, qry)

Then, we can just use the query the same way as usual. 

## Defining our Cohort

In this section, you will begin to explore Kentucky postsecondary degree completions data. As you work through this section, try to ask yourself questions such as:

- What variables are you interested in? 
- What variables do you need to identify the sample you are interested in?
- In which table(s) are these variables available? 
- Are there any missing values in these variables?

As mentioned in the "Learning Objectives" section, we will focus on a specific cohort and track them over time to help answer our overarching question. Recall that in this series of Jupyter notebooks we will use a cohort of postsecondary education degree completers in Kentucky that graduated in the 2013 AY. 

The authoritative source for Kentucky postsecondary degree completions is the `kpeds_degree` table in the `kystats_2020` schema.  This table contains all postsecondary degrees, inclusive of certificates/diplomas through doctoral degrees, conferred at Kentucky 2-year public, 4-year public, and 4-year private postsecondary institutions.

The column `kpeds_degree_year` is the reporting AY of degree conferral.  This column typically corresponds to summer, fall, and spring of the AY such that `kpeds_degree_year` = 2013 would encompass graduates in the summer term of 2012, fall term of 2012, and spring term of 2013. Some postsecondary institutions elect to have multiple summer term sessions within an AY (`kpeds_degree_year`).

>One example of an institution with multiple summer sessions in the 2013 AY is Morehead State University (`kpeds_institution` = '00197600'). Morehead State University will have four viable year/term combinations within the 2013 `kpeds_degree_year` consisting of: (late) summer term of 2012, fall term of 2012, spring term of 2013 and (early) summer term of 2013.

Throughout exploring this dataset, it is important to note that the `coleridge_id` is representative of a unique person throughout the `kystats_2020` schema.  This unique identifier is a modified version of the identifier created within the Kentucky Longitudinal Data System (KLDS). The KLDS matching algorithm has been refined across the years to handle missing values and common data issues across sources (e.g., name changes, short versions of first names, truncated information, transposed digits). Matching individuals outside of the system will require using the hashed `ssn` value contained within the `master_person` table, as you will see in the final section of this notebook.

>Note: Null `ssn` values are also hashed and should not be used when matching outside of the `kystats_2020` schema. We have identified `coleridge_id` values affiliated with hashed null `ssn` values for you in the table `bad_ssn` in the `ada_ky_20` schema.

Here, we will grab all of the columns in the `kpeds_degree` table where the `kpeds_degree_year` is in 2013, and we will add a final column `deg_date` that roughly calculates the date of degree completion. We will use this variable later when finding employment outcomes within a certain time frame, so don't worry about the variable's construction now.

In [ ]:
# sets up the query for everyone graduating in the 2013 academic year
qry = "
select *, case 
    when kpeds_semester = 'Fall' then format('%s-%s-1', kpeds_semester_year, 10)::date
    when kpeds_semester = 'Spring' then format('%s-%s-1', kpeds_semester_year, 4)::date
    when kpeds_semester = 'Summer' then format('%s-%s-1', kpeds_semester_year, 7)::date
    end as deg_date
from kystats_2020.kpeds_degree
where kpeds_degree_year = 2013 and coleridge_id not in (select coleridge_id from ada_ky_20.bad_ssn) 
limit 5
"

# runs this query and displays the results (limited to 5 rows as per the limit statement above)
dbGetQuery(con, qry)

In [ ]:
# removes the limit restriction and assigns the full result to df, a data frame we can work with in R
# read everyone graduating in the 2013 academic year into R
qry = "
select *, case 
    when kpeds_semester = 'Fall' then format('%s-%s-1', kpeds_semester_year, 10)::date
    when kpeds_semester = 'Spring' then format('%s-%s-1', kpeds_semester_year, 4)::date
    when kpeds_semester = 'Summer' then format('%s-%s-1', kpeds_semester_year, 7)::date
    end as deg_date
from kystats_2020.kpeds_degree
where kpeds_degree_year = 2013 and coleridge_id not in (select coleridge_id from ada_ky_20.bad_ssn)
"

df <- dbGetQuery(con, qry)

Let's see if anyone in our cohort had multiple graduations (different degree levels, different semesters) within this time frame by finding the number of graduations and the number of graduates in `df`.

In [ ]:
# count total number of graduations and total number of graduates
df %>%
    summarize(
        graduations = n(),
        graduates = n_distinct(coleridge_id)
    )

## Double-Checking our Cohort

Since the number of graduations is higher than the number of graduates, it is clear that some individuals graduated with multiple degrees, potentially at different levels or in different semesters, within the 2013 academic year. At this point, we have to decide whether our unit of analysis is a person or a credential (degree). Since the outcome of interest is individual employment outcomes, each row should represent a single individual. One method of compressing degree information is to select the highest degree attainment per person within a given AY. This will also make sure we do not overweight, or double count these individuals in future analyses.

We will do so by using SQL's `DISTINCT ON` command, which allows us to ensure that each entry is a single row. In the off chances that an individual has two separate graduations in different semesters each attaining the same degree level (i.e. Associate's), we will take their most recent graduation, as indicated in our `ORDER BY` clause.


> Many repeat completers are at the certificate level. Since many certificates require less than a year for completion, this is an area where multiple completions are both expected and common. When multiple credentials of different levels (`degreerank`) are awarded within the same year, this is commonly a combination of Associate's or Diplomas with certificates,

In [ ]:
# let's get highest graduation now
qry = "
select DISTINCT ON (coleridge_id)
*, case 
    when kpeds_semester = 'Fall' then format('%s-%s-1', kpeds_semester_year, 10)::date
    when kpeds_semester = 'Spring' then format('%s-%s-1', kpeds_semester_year, 4)::date
    when kpeds_semester = 'Summer' then format('%s-%s-1', kpeds_semester_year, 7)::date
    end as deg_date
from kystats_2020.kpeds_degree
where kpeds_degree_year = 2013 and coleridge_id not in (select coleridge_id from ada_ky_20.bad_ssn)
ORDER BY coleridge_id, degreerank, deg_date desc
limit 5
"
dbGetQuery(con, qry)

In [ ]:
# let's read highest graduation into R and assign it to a data frame called highest
qry = "
select DISTINCT ON (coleridge_id)
*, case 
    when kpeds_semester = 'Fall' then format('%s-%s-1', kpeds_semester_year, 10)::date
    when kpeds_semester = 'Spring' then format('%s-%s-1', kpeds_semester_year, 4)::date
    when kpeds_semester = 'Summer' then format('%s-%s-1', kpeds_semester_year, 7)::date
    end as deg_date
from kystats_2020.kpeds_degree
where kpeds_degree_year = 2013 and coleridge_id not in (select coleridge_id from ada_ky_20.bad_ssn)
ORDER BY coleridge_id, degreerank, deg_date desc
"
highest <- dbGetQuery(con, qry)

Just to confirm that our query above selected each `coleridge_id` once, we can verify if the number of rows is the same as the number of distinct `coleridge_id` values found in `highest`.

In [ ]:
# sanity check that number of rows is equal to the number of distinct coleridge_id values
highest %>%
    summarize(
        graduations = n(),
        graduates = n_distinct(coleridge_id)
    )

To see which semesters have the most amount of graduates, we can `count()` the number of graduates by semester of highest degree completion in the 2013 AY.

In [ ]:
#graduates by semester
highest %>%
    count(kpeds_semester) %>%
    arrange(desc(n))

Finally, just to make sure we do not have any misplaced entries, let's make sure that each of the semester/year combinations make sense given the definition of our cohort.

In [ ]:
#graduates by semester/year
highest %>%
    count(kpeds_semester, kpeds_semester_year) %>%
    arrange(desc(n))

Do you notice any entries that seem like they do not belong? Given that there are just a few of these entries, chances are these are simply data input errors, so we will treat them as such. Let's filter out these graduates that should not be in this cohort.

In [ ]:
#graduates by semester/year
highest <- highest %>%
    filter(kpeds_semester_year > 2011, !(kpeds_semester_year == 2012 & kpeds_semester == 'Spring'))

Now we (hopefully) have a data frame that properly represents our cohort.

## Understanding Our Graduates

How can we leverage our data to better understand our cohort of graduates? We will make use of our previously-defined cohort `highest` to answer these questions:

1. How many graduates are there by highest degree rank (within the 2013 AY)?
1. How many graduates are there by institution type? 
1. What are the percentages of graduates who received their primary degrees within the seven major groups? Does this differ by institution location?
1. How many graduates double majored in their highest conferred degree? 
1. Which institutions had the most amount of graduates?
1. Which counties of origin had the most amount of graduates?

<font color=green><h4>Question 1: How many graduates are there by degree group? </h4></font> 

To answer this question, we can simply count the number of rows of each `degreerank`.

In [ ]:
# count of graduates by degree
highest %>%
    count(degreerank)

<font color=green><h4>Question 2: How many graduates are there by institution type? </h4></font> 

Instead of focusing on `degreegroup`, we should be able to look at `kpeds_sector` to answer this question. Let's just make sure that the `kpeds_sector_code` column matches with `kpeds_sector`.

In [ ]:
# see if each sector code corresponds to a specific sector
highest %>%
    count(kpeds_sector_code, kpeds_sector)

Note that AIKCU should fall into the same sector as 4 year independent institutions, which both coincidentally correspond to `kpeds_sector_code` = 3. Let's set all rows where `kpeds_sector` is AIKCU to 4 year independent and then count the number of entries by `kpeds_sector`.

In [ ]:
# AIKCU is the same as 4 year independent (check kpeds_sector_code) so just mutate and count
highest %>%
    mutate(kpeds_sector = ifelse(kpeds_sector == "AIKCU", "4 year independent", kpeds_sector)) %>%
    count(kpeds_sector)

<font color=green><h4>Question 3: What are the percentages of graduates who received their primary degrees within the seven major groups? Does this differ by institution location?</h4></font> 

The Classification of Instructional Programs (CIP) codes are a standardized taxonomic system for identifying the content of a degree program. CIP codes can be used in 2-digit, 4-digit, or 6-digit form, with each 2-digit addition providing more specificity. Using the broadest meaningful category, 2-digit CIP, allows for thirty-nine unique major categories. This presented a challenge for reporting outcomes by major. To balance specificity and utility when visualizing and exploring postsecondary data, the Kentucky Council on Postsecondary Education (CPE) grouped these thirty-nine 2-digit CIP codes into the 7 major groups used in the Kentucky Postsecondary Feedback Report.

Our first step is to replicate the groupings found in the Kentucky Postsecondary Feedback Report. We start by creating a new variable `deg_class` that groups 2-digit CIP codes into 7 larger major groups.

In [ ]:
# add in major groups column
highest <- highest %>%
    mutate(deg_class = case_when(
        substring(kpeds_major1_cip, 1, 2) %in% c('05', '16', '23', '24', '30', '38', '39', '50', '54') ~ "Arts and Humanities",
        substring(kpeds_major1_cip, 1, 2) %in% c('09', '10', '52') ~ "Business",
        substring(kpeds_major1_cip, 1, 2) == '51' ~ "Health",
        substring(kpeds_major1_cip, 1, 2) == '13' ~ "Education",
        substring(kpeds_major1_cip, 1, 2) %in% c('19', '22', '25', '31', '42', '44', '45') ~ "Social and Behavioral Sciences and Human Services",
        substring(kpeds_major1_cip, 1, 2) %in% c('01', '03', '04', '11', '14', '15', '26', '27', '28', '40', '41') ~ "STEM",
        substring(kpeds_major1_cip, 1, 2) %in% c('12', '33', '43', '46', '47', '48', '49') ~ "Trades"
    )
          )

Now, we can use our new variable,`deg_class`, to find the percentage of graduates that fall into each of these seven groups. Recall that this table (`highest`) only contains one row per person. 

In [ ]:
highest %>%
    count(deg_class) %>%
    mutate(pct = round((n/sum(n)) * 100, 2)) %>%
    arrange(desc(pct)) %>%
    select(-n)

Now that we have figured out a way to look at the overall percentage of degrees by major group, we can start to explore whether there any institutional or geographic differences in degree attainment.

There are many potential ways to categorize a postsecondary institution. The table `kpeds_inst_xwalk` has information on each institution, such as: type of institution (`sector`), the broad geographic location (`wib`), and classification of the county of the institution's primary campus (`appalachian`). The query below allows you to see the fields in this crosswalk.

In [ ]:
# see institution crosswalk for appalachian status
qry <- "
select * 
from kystats_2020.kpeds_inst_xwalk 
limit 5
"
dbGetQuery(con, qry)

Let's pull this crosswalk into our environment as a data frame named `inst_xwalk`.  This will allow us to join the institution crosswalk to our previously created `highest` data frame. Combining this into one data frame will allow us to group individual-level information by institutional characteristics.

In [ ]:
# read kpeds_inst_xwalk into R
qry <- "
select *
from kystats_2020.kpeds_inst_xwalk
"
inst_xwalk <- dbGetQuery(con, qry)

We can join the information from `highest` to that of `inst_xwalk` based on the institution codes, which are consistent across the two tables. You may have noted that the institution code has different column names within the two tables, but we can still use the `tidyverse`'s `left_join` function to join the columns from the two tables on the institution code.

In [ ]:
# can match on the institution code
edu_match <- highest %>% 
    left_join(inst_xwalk, by=c("kpeds_institution" = "inst_code"))

#check to see if everything looks right on the surface
head(edu_match)

Right now we are just exploring this dataset to see the possibilities and understand cross-data linkages.  Either `wib` or `appalachian` are convenient and potentially useful measures of institutional geography. 

**Information about these variables:**

- `wib`: WIB is the abbreviation for Workforce Innovation Boards.  This column groups the 120 Kentucky counties into 10 distinct geographic areas (frequently termed Local Workforce Areas [LWAs]).  Counties grouped together within an LWA tend to have similar economic interests and align to areas of frequent cross-county home-work commutes.

- `appalachian`: Counties can be considered Appalachian or not. The Appalachian area has been of great interest for economic development since, at least, the Johnson era. Program initiatives in Kentucky often focus on increasing college-going in Appalachian counties due to the lower overall socioeconomic status (SES).  Program examples include Kentucky College Coaches and GEAR UP, both of which focus on assisting low income and/or first generation students in navigating the transition to postsecondary education.

Let's start by looking at the major group (`deg_class`) by county type (`appalachian`).  There are many ways to organize this depending on the research question.  For now, let's just look at major groups descriptively, first creating separate data frames for all institutions located in Appalachian counties and non-Appalachian counties, and then combining their columns together to allow for a side-by-side comparison.

In [ ]:
# counts of graduates by major group for colleges located in Appalachian counties
appalachian <- edu_match %>%
    filter(appalachian == 1) %>%
    count(deg_class) %>%
    mutate(pct = round((n/sum(n)) * 100, 2)) %>%
    select(-n)
                

# counts of graduates by major group for colleges located in non-Appalachian counties
nonappalachian <- edu_match %>%
    filter(appalachian == 0) %>%
    count(deg_class) %>%
    mutate(pct = round((n/sum(n)) * 100, 2)) %>%
    select(-n)

#binding these two tibbles to look for differences descriptively
cbind(appalachian %>% mutate(college_location = "Appalachian"), 
      nonappalachian %>% mutate(college_location = "Non-Appalachian"))

<font color=green><h4>Question 4: How many graduates double majored in their highest conferred degree?  </h4></font> 

To find double majors, we can see if they have non null `kpeds_major2` values.

In [ ]:
# amount of double majors within highest degree
# a non-null value for kpeds_major2 signifies that the individual had two majors affiliated with this degree
highest %>%
    filter(!is.na(kpeds_major2)) %>%
    count()

<font color=green><h4>Question 5: Which institutions had the most amount of graduates? </h4></font> 

Instead of using institution codes to find the institutions with the most amount of graduates, we can access the common names of these institutions using `kpeds_instname`.

In [ ]:
# counts by institution during the 2013 AY (degree_year)
highest %>%
    count(kpeds_instname) %>%
    arrange(desc(n)) %>%
    head(10)

It is not surprising that our top two institutions are also located in two of our most populous counties (Fayette and Jefferson respectively). Additionally, the University of Kentucky and the University of Louisville are the only research institutions in the state.  

Two of the three Community and Technical Colleges making the top ten are also located in Fayette and Jefferson counties, Bluegrass Community and Technical College and Jefferson Community and Technical College, respectively. 

Lastly, we'll take a look at the breakdown of graduates by in-state county of origin.

<font color=green><h4>Question 6: Which counties of origin had the most amount of graduates? </h4></font> 

To find the counties of origin for our graduates, we will bring in another table in the `kystats_2020` schema, `kpeds_enrollments`. This table provides information on all postsecondary enrollments at Kentucky institutions, as well as details such as student classification, degree sought, and declared major for these students.

In [ ]:
# see kpeds_enrollments
qry <- "
select *
from kystats_2020.kpeds_enrollments
limit 5
"
dbGetQuery(con, qry)

In [ ]:
# read kpeds_enrollments into R
qry <- "
select *
from kystats_2020.kpeds_enrollments
where kpeds_semesteryear in (2012, 2013)
"
enroll <- dbGetQuery(con, qry)

Both `kpeds_countyoforigin` and `kpeds_statecountryoforigin` derive from the same Federal Information Processing Standards (FIPS) codes. `kpeds_statecountryoforigin` corresponds to the 2-digit state/country code while `kpeds_countyoforigin` corresponds to the 3-digit county code. These two columns must be used together to prevent misinterpretation of county location.

>We restrict the state (`kpeds_statecountryoforigin`) to Kentucky to prevent misrepresentation of location.  For example, these prevents us from accidentally attributing a student from Adair County, Indiana to Adair County, Kentucky.

In [ ]:
# find top 10 counties in Kentucky
highest %>% 
    left_join(enroll, "coleridge_id") %>%
    filter(kpeds_statecountryoforigin == "Kentucky") %>%
    group_by(kpeds_countyoforigin) %>%
    summarize(n = n_distinct(coleridge_id)) %>%
    arrange(desc(n)) %>%
    head(10)

<font color=red><h3> Checkpoint 2: WIB and Origin State/Country Breakdowns </h3></font> 

Instead of answering Question 3 using the Appalachian status of each institution, compare the percentage of graduates in a degree group by WIB. Then, find the most popular origin states/countries of our cohort. Are you surprised by this breakdown?


>When joining enrollments back to the degree table, some individuals have a null or NA county of origin. This occurs for two reasons: 1) the left join preserves all records from `highest` regardless of whether a corresponding record exists in enrollments and 2) a legitimate disconnect can exist between the year of degree conferrance and enrollments. Degrees may be held due to unpaid library fines or parking tickets as well as more substantive reasons such as the actual date of a degree defense and the rules of a graduate school concerning the final valid day for a defense in a given semester.  

In [ ]:
# popular degree groups (deg_class) by WIB



In [ ]:
# top 10 most popular origin states



## Finding our Cohort's Employment Records

Now that we have a bit of a better grasp on our cohort, let's find their post-graduation wage outcomes. This will require us to map graduates from our `highest` data frame to employment records. For the purposes of this series of notebooks, we will focus on employment outcomes up to one year after graduation, starting the quarter after graduation.

For each postsecondary graduation, we have AY (`kpeds_degree_year`), calendar year (`kpeds_semester_year`), and term (`kpeds_semester`). The AY is used for reporting purposes, but the calendar year and term are more useful when trying to determine post-graduation employment. Individual institutions have some flexibility in determing the exact date corresponding to the end of a term; however, these terms can still be mapped to the expected quarter of graduation.

**Graduation Calendar Year and Term (table:`kpeds_degree`) translated to Calendar Year and Quarter for Employment (table: `ui_wage_record`):**
- 2012 Summer: Graduated 2012Q3
- 2012 Fall:   Graduated 2012Q4
- 2013 Spring: Graduated 2013Q2
- 2013 Summer (unique to Morehead State University): Graduated 2013Q3

You can see the number of graduates by calendar year and semester by running the code cell below.

In [ ]:
#graduates by semester/year
highest %>%
    count(kpeds_semester, kpeds_semester_year) %>%
    arrange(desc(n))

Let's verify that all graduates in Summer 2013 were from Morehead State University.

In [ ]:
# see institution name for 2013 Summer graduates
highest %>%
    filter(kpeds_semester_year == 2013, kpeds_semester == 'Summer') %>%
    count(kpeds_instname)

This references the additional summer session unique to Morehead State University.  For all other institutions, there are only three viable semester year/term combinations within a given AY.

Since we will perform this join in SQL and will need to call our cohort table in our future work, we use the code below to write a permanent table `cohort` into the `ada_ky_20` schema. This code is identical to the one that created `highest`, except we added in two extra conditions to our `WHERE` clause to eliminate graduates with data entry issues. We also added in `deg_class` (7 major groupings based on 2-digit CIP code) to supplement `kpeds_major1_cip`.

    create table ada_ky_20.cohort as
    select distinct on (coleridge_id)
    *, case 
        when kpeds_semester = 'Fall' then format('%s-%s-1', kpeds_semester_year, 10)::date
        when kpeds_semester = 'Spring' then format('%s-%s-1', kpeds_semester_year, 4)::date
        when kpeds_semester = 'Summer' then format('%s-%s-1', kpeds_semester_year, 7)::date
        end as deg_date,
    case 
        when substring(kpeds_major1_cip, 1, 2) in ('05', '16', '23', '24', '30', '38', '39', '50', '54') then 'Arts and Humanities'
        when substring(kpeds_major1_cip, 1, 2) in ('09', '10', '52') then 'Business'
        when substring(kpeds_major1_cip, 1, 2) in ('51') then 'Health'
        when substring(kpeds_major1_cip, 1, 2) in ('13') then 'Education'
        when substring(kpeds_major1_cip, 1, 2) in ('19', '22', '25', '31', '42', '44', '45') then 'Social and Behavioral Sciences and Human Services'
        when substring(kpeds_major1_cip, 1, 2) in ('01', '03', '04', '11', '14', '15', '26', '27', '28', '40', '41') then 'STEM'
        when substring(kpeds_major1_cip, 1, 2) in ('12', '33', '43', '46', '47', '48', '49') then  'Trades'
        end as deg_class
    from kystats_2020.kpeds_degree
    where kpeds_degree_year = 2013 and coleridge_id not in (select coleridge_id from ada_ky_20.bad_ssn)
          and kpeds_semester_year > 2011 and not (kpeds_semester_year = 2012 and kpeds_semester ='Spring')
    order by coleridge_id, degreerank, deg_date desc

We will join our `cohort` table to Kentucky's UI wage records. Before doing so, let's take a peek at the `ui_wage_record` table located in the `kystats_2020` schema.

In [ ]:
# look at ui wage records in kentucky
qry = "
select *
from kystats_2020.ui_wage_record
limit 5
"
dbGetQuery(con, qry)

First, just to speed up the eventual join as well as add in the `job_date` column, we can create a permanent subset version of our UI wage records, where we only focus on wages before 2015. This subset will work for us since we are limiting our focus to employment outcomes up to one year post-graduation, which in the most extreme cases will end in 2014.

> We will create the `job_date` column in the same fashion as we did for `deg_date`. `format()` works by structuring a variable in a specific fashion, with placeholders that can be represented in this case by `'%s` that will be filled with the following arguments. Here, we will create a date with YYYY-MM-01, with the MM determined by multiplying the quarter by 3 and then subtracting two. Therefore, the potential months corresponding to the quarters will be January, April, July, and October.

    create table ada_ky_20.ky_wages_sub as 
    select *, format('%s-%s-1', calendaryear, qtr*3-2)::date as job_date
    from kystats_2020.ui_wage_record
    where calendaryear < 2015

From here, we can create our permanent table of our cohort's employment outcomes up to one year post graduation by joining our `cohort` table to `ky_wages_sub` based on the common `coleridge_id`, and then only looking for employment up to one year post `deg_date`. We can do so using `'1 year'::interval`, which will specify a specific date interval we can only use on columns of date type. Note that we are only selecting these specific columns out of convenience, and could use `a.*` and `b.*` if we desired, and that we also included `time_after_grad` as the number of days between the graduation and job.

    create table ada_ky_20.cohort_wages as
    select a.coleridge_id, a.degreegroup, a.degreerank, a.kpeds_major1, a.kpeds_major1_cip, a.kpeds_instname, a.kpeds_institution, a.kpeds_sector, a.deg_date, a.deg_class, b.industry, b.majorindustry, b.wages, b.fein, b.employeeno, b.job_date, (b.job_date - a.deg_date) as time_after_grad
    from ada_ky_20.cohort a
    left join ada_ky_20.ky_wages_sub b
    on a.coleridge_id = b.coleridge_id
    where b.job_date > a.deg_date AND (a.deg_date + '1 year'::interval) >= b.job_date

## Understanding Post-Graduation In-State Employment and Earnings

Now that we have matched the Kentucky wage records in the database to our cohort up to one year post graduation, we are going to use this table to get an initial understanding of in-state post-graduation employment and earnings.  

**This section will answer the following questions about in-state employment:**

1. How many individuals and what percentage of our cohort was employed in at least one quarter in the following year after graduation? How does this differ by degree rank?
1. How do annual earnings post-graduation differ by degree rank?
1. How do quarterly earnings post-graduation differ by degree rank?
1. What are the percentage breakdowns for our cohort within Kentucky's five key sectors? How does this differ by institution location? 
1. What is the percentage of our employed cohort that had earnings above minimum wage in at least one quarter? How does this percentage differ by degree rank?

First, before we start to answer these questions, we need to pull our `cohort_wages` table into R.

In [ ]:
# read into R
qry = "
select *
from ada_ky_20.cohort_wages
"
df_wages <- dbGetQuery(con, qry)

In [ ]:
#check the results of this, if our interval worked correctly we will have 4 possible quarters (job_date) for each graduation year/term combination (unique deg_date)
df_wages %>%
   group_by(deg_date) %>%
   summarize(ct_degree_Date = n_distinct(job_date))

#checking the way that the date shifts and how many distinct people are employed from our cohort in a given quarter
df_wages %>%
   group_by(deg_date) %>%
   mutate(total_grads_in_ui_within_the_year = n_distinct(coleridge_id)) %>%
   ungroup() %>%
   group_by(deg_date, job_date)%>%
   summarize(total_grads_in_ui_within_the_year = min(total_grads_in_ui_within_the_year),
              grads_employed_this_qtr          = n_distinct(coleridge_id)) %>%
   arrange(deg_date, job_date)

<font color=green><h4>Question 1: How many individuals and what percentage of our cohort was employed in at least one quarter in the following year after graduation? How does this differ by degree rank? </h4></font> 

If someone in our cohort was employed in at least one quarter in the following year after graduation, they will be in `df_wages`, so we just need to count the number of `coleridge_id` values.

In [ ]:
# amount of people with at least one quarter of earnings
df_wages %>%
    summarize(n=n_distinct(coleridge_id))

To find this percentage, we can divide the count above by the count of distinct `coleridge_id` values in `highest`. For a "fancy" output, we can use the `percent()` function from `scales` to find this percentage.

> The `accuracy` argument in `percent()` will determine the degree of rounding in the output.

In [ ]:
# percentage with at least one quarter of earnings
df_wages %>%
    summarize(percent = percent(n_distinct(coleridge_id) /n_distinct(highest$coleridge_id), accuracy=.01))

We can break this down by `degreerank` by finding these counts by `degreerank` and dividing by counts from `highest`. To divide counts by `degreerank` within the two dataframes, we can join them based on the `degreerank` columns and divide them from there.

In [ ]:
# percentage with at least one quarter of earnings by group

#using wages, how many people were able to be found in at least one wage quarter of the year following graduation, split by degreegroup
wages_group <- df_wages %>% 
    group_by(degreegroup) %>% 
    summarize(n_wages = n_distinct(coleridge_id)) %>% 
    ungroup()

#using highest degree, how many graduates were there by degreegroup
full_group <- highest %>% 
    group_by(degreegroup) %>% 
    summarize(n_full = n_distinct(coleridge_id)) %>%
    ungroup()

#starting with summary information from highest degrees
#add in summaries from the wage table to calculate percent with at least one quarter of wages by degreegroup
full_group %>% 
    left_join(wages_group, "degreegroup") %>% 
    mutate(pct = percent(n_wages/n_full, accuracy=.01)) %>% 
    select(degreegroup, pct) %>%
    arrange(desc(pct))


**There are several options for grouping degrees:**

- `degreegroup`: A KYSTATS derived column that compresses `kpeds_degreelevelshortdesc` into four large groups.

- `degreerank`: Another KYSTATS derived column, which compromises between full information and `degreegroup`.  There are 8 groupings in `degreerank`. This column is used to determine highest degree for the Postsecondary Feedback Report (PSFR) produced by KYSTATS in partnership with CPE.  
>The `CASE WHEN` statement provides an additional grouping method for degrees as implemented in the PSFR. `degreecategory_psfr` is not a permanent column, but provides a code chunk that can be used if you wanted to replicate and expand upon this report. 

- `kpeds_degreelevelshortdesc`: Used for reporting purposes, this column has 15 unique values.  This column can be used in tandem with `kpeds_degreelevellongdesc` to clarify the certificates conferred.

Let's look at these potential grouping variables below:

In [ ]:
degree_lvls_qry <-"SELECT distinct degreegroup, 
                          degreerank, 
                          case when degreerank in (1,2) then 'Doctoral'
                               when degreerank = 3      then 'Master'
                               when degreerank in (4,5) then 'Bachelor'
                               when degreerank = 6      then 'Associate'
                               when degreerank = 7      then 'Diploma'
                               when degreerank = 8      then 'Certificate'
                               else null end as degree_category_psfr,
                          kpeds_degreelevelshortdesc
                   FROM kystats_2020.kpeds_degree
                   ORDER BY degreerank, degreegroup, kpeds_degreelevelshortdesc;"

dbGetQuery(con, degree_lvls_qry)

<font color=green><h4>Question 2: How do annual earnings post-graduation differ by degree rank? </h4></font> 

We may also be curious about how annual wages differ based on the type of highest degree conferred. 


Let's combine `degreerank` with `coleridge_id` to look at wage variation by degree and person in our `group_by()`. Recall that the `coleridge_id` grouping is more granular than `degreerank` since we only have highest degree earned in the 2013 AY. However, we still need to include `degreerank` in our `group_by()` statement if we want to output a table with the associated `degreerank` column.

>Each person has only one `degreerank`, but each `degreerank` can have more than one person.

In [ ]:
# show wages by person and degreerank
df_wages %>%
    group_by(coleridge_id, degreerank) %>%
    summarize(total_wages = sum(wages)) %>%
    head()

Now, we can group this information by `degreerank` and summarize to get an initial understanding of how total (yearly) in-state wages relate to highest degree level.  

>Be aware that we are summing all UI-covered in-state wages and calling this `total_wages`.  Think about the implications of this assumption as you move forward.

In [ ]:
# more nuanced look at distribution
df_wages %>%
    group_by(coleridge_id, degreerank) %>%
    summarize(total_wages = sum(wages)) %>% #this gets us up to the code used above
    ungroup() %>% #we ungroup to get rid of the coleridge_id variable
    group_by(degreerank) %>% #group by only degreegroup to get us a wage summary by this variable
    summarize('.1'  = quantile(total_wages, .1),
              '.25' = quantile(total_wages, .25),
              '.5'  = quantile(total_wages, .5),
              '.75' = quantile(total_wages, .75),
              '.9'  = quantile(total_wages, .9)
             )

At this point, you might be wondering why you haven't seen a visual representation of these earnings. Don't worry, we'll cover data visualization in the next [notebook](04_01_Data_Visualization.ipynb).

<font color=green><h4>Question 3: How do quarterly earnings post-graduation differ by degree rank? </h4></font> 

Let's take a look at the quarterly wage distribution as well to see if we can learn more about our cohort's wage distribution. You will notice that this code is very similar to the code used for total (yearly) wages.  Altering the code to look at a single quarter is going to provide us with a look at wages for a three month period in more detail.

In [ ]:
# more nuanced look at quarterly earnings distribution by group
df_wages %>%
    group_by(coleridge_id, degreerank, job_date) %>% #recall that job_date was created as a convenient way to uniquely identify a calendar year/quarter combination
    summarize(quarterly_wages = sum(wages)) %>% #sums wages from multiple jobs within a quarter for a given person
    ungroup() %>%
    group_by(degreerank) %>%
    summarize('.1'  = quantile(quarterly_wages, .1),
              '.25' = quantile(quarterly_wages, .25),
              '.5'  = quantile(quarterly_wages, .5),
              '.75' = quantile(quarterly_wages, .75),
              '.9'  = quantile(quarterly_wages, .9)
             )

<font color=green><h4>Question 4: What are the percentage breakdowns for our cohort within Kentucky's five key sectors? How does this differ by institution location?  </h4></font> 

To find the most common industries, we can focus on the `majorindustry` column, as compared to the `industry` one. Both of these are derived from the North American Industry Classification System (NAICS), with `majorindustry` pulling from the first 2-digits of the NAICS code and `industry `pulling from the full 6-digit NAICS code. Using `majorindustry` provides additional value by saving real estate when visualizing results. Furthermore, `majorindustry` has been crosswalked by partner agencies (i.e., the Cabinet for Economic Development and the Kentucky Workforce Innovation Board) to the five key sectors for Kentucky. 

>If you want to apply Kentucky Key Sectors to employment data from border states, the following 2-digit NAICS codes fall into each category: <br>
>Advanced Manufacturing, 31-33 <br>
>Business and IT Services, 42, 51, 52, 54 <br>
>Construction and Trades, 23 <br>
>Health Science, 62 <br>
>Transportation and Logistics, 48-49 <br>


In [ ]:
# add in five key sectors
#groups majorindustry and renames to conform with terms used in Kentucky
df_wages <- df_wages %>%
    mutate(key_sect = case_when(
        majorindustry ==    "Manufacturing" ~ "Advanced Manufacturing",                                           
        majorindustry ==    "Construction" ~ "Construction and Trades",                                           
        majorindustry ==    "Health Care and Social Assistance" ~ "Health Science",                              
        majorindustry ==    "Transportation and Warehousing" ~ "Transportation and Logistics",                    
        majorindustry %in% c("Professional, Scientific, and Technical Services",
                             "Finance and Insurance", 
                             "Information",
                             "Wholesale Trade") ~ "Business and IT Services",    
        TRUE ~ "Non Key"
    )
          )

Now we can use `key_sect` to find the percentage of individuals in `df_wages` that were employed for at least one quarter in each of these sectors.

>Note: We have not compressed wages down to one row per person with an affiliated `majorindustry` of attribution. Part of using wages in your research question will be operationalizing primary employment based on the desired time interval of interest.

In [ ]:
# percentages by five key sectors
df_wages %>%
    group_by(key_sect) %>%
    summarize(n = n_distinct(coleridge_id)) %>%
    ungroup() %>%
    mutate(pct = round((n/sum(n)) * 100, 2)) %>%
    select(-n)

Again, by institution location, for this example we will hone in on `appalachian` as opposed to `wib`.

In [ ]:
names(df_wages)

In [ ]:
names(inst_xwalk)

In [ ]:
# can match on the institution code
df_wages_app <- df_wages %>% 
    left_join(inst_xwalk, by=c("kpeds_institution" = "inst_code")) 

In [ ]:
# find percentage employed by sectors for graduates from appalachian-based institutions
top_app <- df_wages_app %>%
    filter(appalachian == 1) %>%
    group_by(key_sect, appalachian) %>%
    summarize(n = n_distinct(coleridge_id)) %>%
    ungroup() %>%
    mutate(pct = round((n/sum(n)) * 100, 2)) %>%
    select(-n)


# find percentage employed by sectors for graduates from non-appalachian-based institutions
top_nonapp <- df_wages_app %>%
    filter(appalachian == 0) %>%
    group_by(key_sect, appalachian) %>%
    summarize(n = n_distinct(coleridge_id)) %>%
    ungroup() %>%
    mutate(pct = round((n/sum(n)) * 100, 2)) %>%
    select(-n)

cbind(top_app, top_nonapp)

<font color=green><h4>Question 5: What is the percentage of our employed cohort that had earnings above minimum wage in at least one quarter? How does this percentage differ by degree rank? </h4></font> 

Since 2010, Kentucky's minimum wage has been $7.25/hour. Assuming that a full quarter is 13 weeks, and that a full work week is 35 hours, we will find the percentage of our cohort that earned more than 7.25\*35\*13 in a quarter.

First, let's find out how much that comes to per quarter.

In [ ]:
# see how much that comes out to be
7.25 * 13 * 35

Now, we can easily find the count of people in our cohort that experienced at least one quarter making above this threshold.

In [ ]:
# with at least one quarter
df_wages %>%
    group_by(coleridge_id, job_date) %>%
    summarize(q_wages = sum(wages)) %>%
    ungroup() %>%
    filter(q_wages > (7.25*13*35)) %>%
    summarize(n=n_distinct(coleridge_id))

We can extend this calculation slightly to find the percentage of our wage earners that had at least one quarter of in-state UI-covered earnings exceeding this threshold within one year of approximate graduation.

In [ ]:
# with at least one quarter
df_wages %>%
    group_by(coleridge_id, job_date) %>%
    summarize(q_wages = sum(wages)) %>%
    ungroup() %>%
    filter(q_wages > (7.25 * 13 * 35)) %>%
    summarize(n = n_distinct(coleridge_id)) %>%
    summarize(pct = percent(n/n_distinct(df_wages$coleridge_id), .01))

We can also find the percentage of our cohort that had earnings greater than this threshold in all four quarters.

In [ ]:
# all four quarters
df_wages %>%
    group_by(coleridge_id, job_date) %>%
    summarize(q_wages = sum(wages)) %>%
    ungroup() %>%
    filter(q_wages > (7.25*13*35)) %>%
    group_by(coleridge_id) %>%
    summarize(n_quarters = n_distinct(job_date)) %>%
    ungroup() %>%
    filter(n_quarters == 4) %>%
    summarize(pct = percent(n_distinct(coleridge_id)/n_distinct(df_wages$coleridge_id), .01)) 

We can then find these counts by `degreerank` by including the variable in the `group_by()` command. After finding counts to compare to the counts for the entire `degreerank` counts in `df_wages`, we can then join the two data frames by `degreerank` and divide their respective counts to find the percentages by group.

>Because these calculations start from df_wages, we are finding the percent exceeding this cutoff out of those with any in-state UI-covered wages in the year post-graduation.  This could be altered to look at percent exceeding this cutoff out of all graduates. 

In [ ]:
# with at least one quarter
by_group <- df_wages %>%
    group_by(coleridge_id, degreerank, job_date) %>%
    summarize(q_wages = sum(wages)) %>%
    ungroup() %>%
    filter(q_wages > (13*7.25*35)) %>%
    group_by(degreerank) %>%
    summarize(n_cutoff = n_distinct(coleridge_id))


# counts for total group
full_group <- df_wages %>%
    group_by(degreerank) %>%
    summarize(n = n_distinct(coleridge_id))

full_group %>% 
    left_join(by_group, "degreerank") %>% 
    mutate(pct = percent(n_cutoff/n, accuracy=.01)) %>% 
    select(degreerank, pct) %>%
    arrange(desc(pct))

In [ ]:
# with all 4 quarters exceeding the minimum threshold
by_group <- df_wages %>%
    group_by(coleridge_id, degreerank, job_date) %>%
    summarize(q_wages = sum(wages)) %>%
    ungroup() %>%
    filter(q_wages > (13*7.25*35)) %>%
    group_by(degreerank, coleridge_id) %>%
    summarize(n_quarters = n_distinct(job_date)) %>%
    ungroup() %>%
    filter(n_quarters == 4) %>%
    group_by(degreerank) %>% 
    summarize(n_cutoff = n_distinct(coleridge_id))


# counts for total group
full_group <- df_wages %>%
    group_by(degreerank) %>%
    summarize(n = n_distinct(coleridge_id))

full_group %>% 
    left_join(by_group, "degreerank") %>% 
    mutate(pct = percent(n_cutoff/n, accuracy=.01)) %>% 
    select(degreerank, pct) %>%
    arrange(desc(pct))

<font color=red><h3> Checkpoint 3: Percentages by WIB </h3></font> 

Instead of answering Question 4 using the Appalachian status of each institution, compare the percentage of graduates within each WIB that find employment by key sectors.

In [ ]:
# employment by WIB

## Finding Employment Records for our Cohort outside of Kentucky

For the last part of this notebook, we will look at part of our cohort that was employed in Ohio by linking our cohort to the Ohio Unemployment Insurance (UI) wage records. After linking Kentucky graduates to Ohio wage records, we will analyze common employment patterns over the four potential quarters of employment post graduation.

### Join Cohort to Ohio's UI Wage Records

First, let's take a look at the Ohio UI wage records data, which is stored in the `data_ohio_olda_2018` schema. There are two potential tables we can use: `oh_ui_wage_by_employer` and `oh_ui_wage_by_quarter`, where the latter is an aggregated version of the former based on the quarter. Since we have been looking at employment outcomes at the employer-level granularity, we will work with the `oh_ui_wage_by_employer` table. Let's take a quick look at it.

In [ ]:
# look at oh_ui_wage_by_employer
qry <- "
select *
from data_ohio_olda_2018.oh_ui_wage_by_employer
limit 5
"
dbGetQuery(con, qry)

Do you notice that there is not a common `coleridge_id` value we can use to join to our cohort like in other tables in the `kystats_2020` schema? Instead, they have hashed `ssn` values. Luckily, we have a table `master_person` in the `kystats_2020` schema that allows us to match `coleridge_id` values to `ssn` values to match these individuals to tables in other schemas.

Let's take a quick peek at `master_person`.

In [ ]:
# look at master_person
qry <- "
select *
from kystats_2020.master_person
limit 5
"
dbGetQuery(con, qry)

For future use, we have created a permanent table `cohort_w_ssns` that shows how we joined our `cohort` table to `master_person` to find the `ssn` values associated with each `coleridge_id`.

    create table ada_ky_20.cohort_w_ssns as
    select a.ssn, b.*
    from kystats_2020.master_person a
    inner join ada_ky_20.cohort b
    on a.coleridge_id = b.coleridge_id
    
We can now call this table in any notebook as long as we initialized our R libraries and created our connection to the `appliedda` database.

At this point, we have the variable (`ssn`) we can use to link our cohort to Ohio's UI wage records to find employment outcomes in Ohio for our cohort. As we did in initially joining our cohort to Kentucky's UI wage records, we will create a column `job_date` that functions exactly like `job_date` in `df_wages` prior to the join. We can also limit the years of Ohio's UI wage records to speed up the eventual join. We can refer to this permanent table as `small_ohio_ui`.

    create table ada_ky_20.small_ohio_ui as
    select *, format('%s-%s-01', year, quarter*3-2)::date job_date 
    from data_ohio_olda_2018.oh_ui_wage_by_employer
    where year in ('2012', '2013','2014','2015')

> Recall that we use an interval to match our cohort by their `deg_date` to this `job_date` to find employment outcomes for only up to one year after graduation.

In [ ]:
# see small_ohio_ui
qry = "
select *
from ada_ky_20.small_ohio_ui
limit 5
"
dbGetQuery(con, qry)

Finally, we can join our cohort to the Ohio UI wage records based on common `ssn` values and where the job date is within one year of the graduation date. We created this as another permanent table that you can refer to in the future as `oh_wages`.

> We tried to follow similar naming conventions as we did for `cohort_wages` for each of the variables.

    create table ada_ky_20.oh_wages as 
    select b.employer::varchar as employeeno, b.wages, b.job_date, a.coleridge_id, a.degreegroup, a.degreerank, 
    a.kpeds_major1, a.kpeds_major1_cip, a.kpeds_instname, a.kpeds_sector, a.deg_date, (b.job_date - a.deg_date) as time_after_grad, 
    'OH'::varchar as state, a.deg_class
    from ada_ky_20.cohort_w_ssns a
    left join ada_ky_20.small_ohio_ui b
    on a.ssn = b.ssn_hash
    where b.job_date > a.deg_date AND (a.deg_date + '1 year'::interval) >= b.job_date

In [ ]:
# see oh_wages table
qry = "
select *
from ada_ky_20.oh_wages
limit 5
"
dbGetQuery(con, qry)

### Combine Ohio and Kentucky Employment Records for our Cohort

Now that we have the `oh_wages` and `cohort_wages` tables, we just need to combine the two into one table. We will do this in SQL, but you can also perform this manipulation in R. Basically, we need the column names to be the same across the two tables. Let's compare the two as of now by taking a look at `cohort_wages` once more.

In [ ]:
# see cohort_wages table
qry = "
select *
from ada_ky_20.cohort_wages
limit 5
"
dbGetQuery(con, qry)

You can see that `oh_wages` is missing columns for `industry`, `majorindustry`, and `fein`, while `cohort_wages` is missing `state`. Let's ignore the three columns in `cohort_wages` that are missing in `ky_wages` and add in `state` before using a `UNION` to join the two tables.

>`UNION` and `UNION ALL` will return the same results in this instance. `UNION` deletes duplicate records while `UNION ALL` maintains all records. The `state` variable ensures that records between these two tables will always have at least one variable difference.

In [ ]:
# try combining cohort_wages and oh_wages
qry = "
select coleridge_id, degreegroup, degreerank, kpeds_major1, kpeds_major1_cip, kpeds_instname, kpeds_sector, deg_date, 
wages, job_date, time_after_grad, deg_class, 'KY'::varchar as state
from ada_ky_20.cohort_wages
UNION ALL
select coleridge_id, degreegroup, degreerank, kpeds_major1, kpeds_major1_cip, kpeds_instname, kpeds_sector, deg_date, 
wages, job_date, time_after_grad, deg_class, state
from ada_ky_20.oh_wages
limit 5
"
dbGetQuery(con, qry)

Let's read this combined table into R as `combined_wages`.

In [ ]:
# try combining cohort_wages and oh_wages
qry = "
select coleridge_id, degreegroup, degreerank, kpeds_major1, kpeds_major1_cip, kpeds_instname, kpeds_sector, deg_date, 
wages, job_date, time_after_grad, deg_class, 'KY'::varchar as state
from ada_ky_20.cohort_wages
UNION ALL
select coleridge_id, degreegroup, degreerank, kpeds_major1, kpeds_major1_cip, kpeds_instname, kpeds_sector, deg_date, 
wages, job_date, time_after_grad, deg_class, state
from ada_ky_20.oh_wages
"

#this is the critical difference- here we assign the results to a data frame in our environment
combined_wages <- dbGetQuery(con, qry)

### Employment Outcomes in Ohio

Now that we have been able find employment records for our cohort in Ohio and combined them with those in Kentucky, let's begin to understand graduates who just found employment in Ohio through another series of guided questions:

1. How many individuals and what percentage of our cohort met the following criteria in the year post-graduation: no in-state employment and at least one quarter of Ohio employment?
1. Which Kentucky institutions had the highest percentage of graduates find employment only in Ohio during the year following degree completion?
1. From which states are these individuals originally?
1. What does the distribution of quarterly earnings look like for this group?


Since we will be focusing solely on those in our original cohort that only had earnings in Ohio, we will filter out all `coleridge_id` values from `combined_wages` that exist in `df_wages`.

In [ ]:
# isolate cohort that only had wages in Ohio during the year after graduation
oh_wages <- combined_wages %>%
    filter(!(coleridge_id %in% df_wages$coleridge_id))

<font color=green><h4>Question 1: How many individuals and what percentage of our cohort met the following criteria in the year post-graduation: no in-state employment and at least one quarter of Ohio employment? </h4></font> 

We can use the same commands as we used in a previous section to find these answers, just switching out the data frame from `df_wages` to `oh_wages`.

In [ ]:
# number of individuals
oh_wages %>%
    summarize(n = n_distinct(coleridge_id))

In [ ]:
# percent of cohort
percent(n_distinct(oh_wages$coleridge_id)/n_distinct(highest$coleridge_id), .01)

<font color=green><h4>Question 2: Which Kentucky institutions had the highest percentage of graduates find employment only in Ohio during the year following degree completion? </h4></font> 

Since postsecondary outcomes are often attributed to the degree-conferring institution, we will look at what percentage of an institution's graduates find employment only in Ohio. A larger institution may have a larger number of individuals employed only in Ohio, but a smaller percentage of the overall graduating class when compared to a smaller institution. 

We will still follow a similar procedure, finding the number of graduates per institution, the number of graduates per institution employed only in Ohio, and then calculating the percentage of graduates this represents.

In [ ]:
# find counts by institution in oh_wages
inst_nums <- oh_wages %>%
    group_by(kpeds_instname) %>%
    summarize(n = n_distinct(coleridge_id))

# total counts by institution
tot_nums <- highest %>%
    group_by(kpeds_instname) %>%
    summarize(n_tot = n_distinct(coleridge_id))

# sort institutions by top percentages
inst_nums %>%
    left_join(tot_nums, "kpeds_instname") %>%
    mutate(pct =(round((n/n_tot)*100, 2))) %>%
    arrange(desc(pct)) %>%
    select(kpeds_instname, pct, n_tot) %>%
    head(10)

Are you surprised by these results? Why or why not?

<font color=green><h4>Question 3: From which states are these individuals originally? </h4></font> 

Earlier, we took a look at the `kpeds_countyoforigin` variable from the `enroll` data frame, as the variable tracks counties of origin for our cohort. Instead, we will want to focus on `kpeds_statecountryoforigin` from `enroll` here.

In [ ]:
# find top 10 states
oh_wages %>% 
    left_join(enroll, "coleridge_id") %>%
    group_by(kpeds_statecountryoforigin) %>%
    summarize(n=n_distinct(coleridge_id)) %>%
    arrange(desc(n)) %>%
    head(10)

>There are very few null (NA) values in the actual `kpeds_statecountryoforigin` column of the `kpeds_enrollments` table.  The null values in the results above occur because we are looking at the earliest `kpeds_degree` data loaded into the ADRF and finding the occasional disconnect between `kpeds_degree` and `kpeds_enrollments`. 

<font color=green><h4>Question 4: What does the distribution of quarterly earnings look like for this group? </h4></font> 

In [ ]:
# quarterly earnings
oh_wages %>%
    group_by(coleridge_id, job_date) %>%
    summarize(quarterly_wages = sum(wages)) %>%
    ungroup() %>%
    summarize('.1' = quantile(quarterly_wages, .1),
              '.25' = quantile(quarterly_wages, .25),
              '.5' = quantile(quarterly_wages, .5),
              '.75' = quantile(quarterly_wages, .75),
              '.9' = quantile(quarterly_wages, .9)
             )

<font color=red><h3> Checkpoint 4: Kentucky's Brain Drain </h3></font> 

What `deg_class` has the highest percentage of these graduates that only found employment in Ohio after completing a credential in Kentucky?
>One area of interest in Kentucky is the `deg_class` containing engineers (`deg_class` == 'STEM'). You can also find engineers specifically using the CIP code. 

In [ ]:
# percentage of degrees by major group who completed in Kentucky but were employed only in Ohio


### Common Employment Patterns

Finally, after combining Ohio employment outcomes with Kentucky employment outcomes for our cohort, we can analyze common employment patterns for our cohort. Since we are analyzing employment outcomes up to one year post-graduation, there are 16 potential patterns to look at. To do so, we will want a data frame with each quarter after graduation as separate indicator columns, and the corresponding counts of the amount of individuals in our cohort with that specific pattern.

Before we start analyzing these employment patterns, we should convert `time_after_grad` to `q_after_grad` so we have uniform quarters after graduation.

In [ ]:
# unique values of time_after_grad
unique(combined_wages$time_after_grad)

In [ ]:
# add in quarter after graduation
combined_wages <- combined_wages %>%
    mutate(q_after_grad = round(time_after_grad/90)) #default rounding behavior rounds to an integer

# see unique values of q_after_grad
unique(combined_wages$q_after_grad)

We still do not have employment patterns for everyone in our initial cohort, though. We can confirm this below.

In [ ]:
# count total number of cohort
highest %>%
    summarize(n=n_distinct(coleridge_id))

In [ ]:
# count number with employment outcomes in Kentucky and/or Ohio
combined_wages %>%
    summarize(n=n_distinct(coleridge_id))

As you can see, we are missing some members of our original cohort, which is due to our `WHERE` clauses in finding these wage outcomes that disregard those who did not show up in the UI wage records. We can solve for this issue by joining `combined_wages` onto `highest` using a `left_join`.

In [ ]:
# employment outcomes for all of those in our original cohort
full_cohort <- highest %>%
    left_join(combined_wages, c("coleridge_id", "degreegroup", "degreerank", "kpeds_major1", 
                                "kpeds_instname", "kpeds_sector", "deg_date"))

In [ ]:
# make sure we have the proper amount of coleridge_id values
full_cohort %>%
    summarize(n = n_distinct(coleridge_id))

Let's see what an entry looks like for someone who did not appear in either of the UI wage records.

In [ ]:
full_cohort %>%
    filter(is.na(wages)) %>%
    head(1)

Before we can find employment patterns for each individual in our cohort, we would like to have four rows per individuals, with each row corresponding to one of the four potential quarters with an indicator tracking if they were employed in that quarter. We can use the `complete()` function to generate these additional rows that do not already exist in the data frame, but first, we need to have potential values of `q_after_grad` to just be 1-4.

In [ ]:
# see potential values of q_after_grad
unique(full_cohort$q_after_grad)

Since those with no employment outcomes have `NA` values for `q_after_grad`, let's just assign `q_after_grad` values for them to 1 so we can use `complete()`.

In [ ]:
# set all where quarter is na equal to 1 so complete uses 1,2,3,4 as the options for quarter
full_cohort$q_after_grad[is.na(full_cohort$q_after_grad)] =1

Now that we have all of our `coleridge_id` values and `q_after_grad` combinations, we can use `complete()` to generate rows where employment outcomes do not currently exist.

In [ ]:
head(full_cohort)

In [ ]:
# need to complete full_cohort and fill out for all four quarters someone doesn't exist
full_cohort <- full_cohort %>%
    complete(coleridge_id, q_after_grad, fill=list(wages=0))

Now we have a complete data frame with at least four entries per person. Let's aggregate on quarter because we only want to look at employment patterns based on quarter, not quarter and employer.

In [ ]:
# aggregate by quarter
by_quarter <- full_cohort %>%
    group_by(coleridge_id, q_after_grad) %>%
    summarize(tot_wages = sum(wages)) %>%
    ungroup()

In [ ]:
# see by_quarter
head(by_quarter)

As a sanity check, we can make sure we have four rows for every `coleridge_id`.

In [ ]:
# confirm that we have four entries per person
nrow(by_quarter) == (n_distinct(by_quarter$coleridge_id) * 4)

Now, instead of having wages corresponding to each `coleridge_id`/`quarter`, we can add an indicator variable tracking if an individual had earnings greater than 0 in a given quarter.

In [ ]:
# add new indicator column for if wages are greater than 0
# then also get rid of the actual wages
by_quarter <- by_quarter %>%
    mutate(wage_ind = ifelse(tot_wages == 0, "0", "1")) %>%
    select(-tot_wages)

In [ ]:
# see wage indicator variable
head(by_quarter)

At this point, we have all of the information we need to generate our desired data frame, with the counts of each pattern. We will need to widen our data frame, which we will be able to do using `pivot_wider()`. First, let's rename the `q_after_grad` values to `q1`-`q4`.

In [ ]:
# now need to expand so that each quarter is a row 
# but first, need to expand a character/factor, not a numerical column
by_quarter <- by_quarter %>%
    mutate(new_quarter = case_when(
    q_after_grad == 1 ~ "q1",
    q_after_grad == 2 ~ "q2",
    q_after_grad == 3 ~ "q3",
    q_after_grad == 4 ~ "q4")) %>%
    select(-q_after_grad)

In [ ]:
# see new column
head(by_quarter)

Now, we can leverage the `pivot_wider()` function, taking the new names for the columns from the values in `new_quarter` and populating these columns with the corresponding entries in `wage_ind`. 

In [ ]:
# use pivot_wider
wage_by_q <- by_quarter %>% 
    pivot_wider(names_from = new_quarter, values_from = wage_ind)

# see wage_by_q
head(wage_by_q)

Finally, we just need to aggregate based on the combination of patterns!

In [ ]:
patterns <- wage_by_q %>%
            group_by(q1,q2,q3,q4) %>%
            summarise(count = n_distinct(coleridge_id)) %>%
            arrange(desc(count)) %>%
            ungroup()

In [ ]:
head(patterns)

Just to make this a bit clearer, we can change our indicator from 1 and 0 to "Yes" and "No".

In [ ]:
patterns <- patterns %>%
            mutate(q1 = ifelse(q1 == 1, "Yes", "No")) %>%
            mutate(q2 = ifelse(q2 == 1, "Yes", "No")) %>%
            mutate(q3 = ifelse(q3 == 1, "Yes", "No")) %>%
            mutate(q4 = ifelse(q4 == 1, "Yes", "No")) 

In [ ]:
patterns

To add some finishing touches, we can count the percentage of our cohort by employment pattern as well.

In [ ]:
patterns <- patterns %>%
    mutate(pct = percent(count/sum(count),.01))

In [ ]:
patterns

Isn't this cool? We can save it as a .csv for future use.

In [ ]:
# write to csv
write.csv(patterns, "/nfshome/INSERT_USERNAME/patterns.csv")

In this notebook, you have covered how to identify the cohort that you are interested in from a database and save it as data frame in R. You have also seen how to conduct descriptive analyses in R, such as checking missing values and breaking down the sample based on variables of interest.

After you find interesting results, you may want to present them in the form of pictures, or visualizations. In the next notebook, which will cover [Data Visualization](Data_Visualization.ipynb), we will show you how to leverage more of the `tidyverse` suite of packages in R to display some of your findings.